In [3]:
import h5py
import random
from torch.utils.data import Dataset

class GPTDataset(Dataset):
    def __init__(self, dataset_path: str, seq_len: int, window_size: int, rng: random.Random):
        self.dataset = h5py.File(dataset_path, 'r')['token_ids']
        self.seq_len = seq_len
        self.window_size = window_size
        self.rng = rng
        self.size = len(self.dataset)

    def __len__(self):
        return self.size
    
    def __getitem__(self.idx):
        start_point = self.rng.randint(0, self.size) # 0부터 self.size까지 중 랜덤
        end_point = min(start_point + (window_size + 1), self.size) # 최소 start_point에서 window_size+1만큼 더한거부터 self.size까지
        window = self.dataset[start_point:end_point]

        input_ids = window[:-1]
        label_ids = window[1:]
        return input_ids, label_ids

### getitem 함수
- start_point : 0 ~ 데이터셋 길이 사이의 임의의 시작점을 고른다.
- end_point : 한 sample의 끝점을 고른다.
- window : window size + 1개의 연속된 토큰을 슬라이싱한다.
- input_ids : 첫 번째 토큰부터 window_size개 = Input Ids
- label_ids : 두 번째 토큰부터 window_size개 = Label Ids

모델에 실제로 input으로 들어가게 되는 input ids와 label ids는 (batchsize, seq_length)의 shape을 가짐<br>
모델에는 input ids가 들어가고 출력된 logits과 label ids와 비교하게 된다.<br>
seq_length는 window_size와 동일 

In [ ]:
import torch

def create_future_mask(x: torch.Tensor, offset: int = 0) -> torch.Tensor:
    seq_len = x.size(-1)

    # Create shifted upper triangular matrix
    future = torch.ones((seq_len, seq_len), dtype = torch.bool, device = x.device)
    future = future.triu(1)

    future_mask = future.view((1, ) * (x.ndim - 1) + future.size())

    return future_mask.expand(x.shape + future_mask.shape[-1:]) # (b, s, s) 